<a href="https://colab.research.google.com/github/otamajakusi/wav2vec2/blob/main/wav2vec2_ja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install datasets==1.13.3
!pip install transformers==4.11.3
!pip install torchaudio
!pip install librosa
!pip jiwer

In [ ]:
%%capture
# If you don't have jsut package, download and extract it.
# https://sites.google.com/site/shinnosuketakamichi/publication/jsut
# !wget http://ss-takashi.sakura.ne.jp/corpus/jsut_ver1.1.zip
# !unzip jsut_ver1.1.zip -d /content/drive/MyDrive/

In [ ]:
!cp -r /content/drive/MyDrive/jsut_ver1.1/basic5000/wav .

In [ ]:
!git clone https://github.com/sarulab-speech/jsut-label

Cloning into 'jsut-label'...
remote: Enumerating objects: 9628, done.
remote: Counting objects: 100% (9628/9628), done.
remote: Compressing objects: 100% (8841/8841), done.
remote: Total 9628 (delta 4598), reused 5805 (delta 784), pack-reused 0
Receiving objects: 100% (9628/9628), 8.61 MiB | 13.18 MiB/s, done.
Resolving deltas: 100% (4598/4598), done.


In [ ]:
import yaml
from pathlib import Path
from datasets import Dataset

def create_dataset(yaml_file, wav_dir: Path):
    paths = []
    texts = []

    with open(yaml_file) as f:
        meta = yaml.safe_load(f)

    for i, m in enumerate(meta.keys()):
        wav = wav_dir / m
        paths.append(f"{wav}.wav")

        sentence = meta[m]["phone_level3"]
        sentence = sentence.replace("-pau", " ")
        sentence = sentence.replace("-", "")
        sentence = sentence.lower() + " "
        texts.append(sentence)
    data = Dataset.from_dict({"path": paths, "text": texts})
    data = data.shuffle(seed=42)
    return data.train_test_split(test_size=0.1, seed=42)

In [ ]:
dataset = create_dataset("/content/jsut-label/text_kana/basic5000.yaml", Path("wav"))
dataset

DatasetDict({
    train: Dataset({
        features: ['path', 'text'],
        num_rows: 4500
    })
    test: Dataset({
        features: ['path', 'text'],
        num_rows: 500
    })
})

In [ ]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = train_dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=train_dataset.column_names,
)
vocab_test = test_dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=test_dataset.column_names,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
vocab_dict

{'[PAD]': 26,
 '[UNK]': 25,
 'a': 13,
 'b': 12,
 'c': 14,
 'd': 21,
 'e': 10,
 'f': 3,
 'g': 8,
 'h': 6,
 'i': 16,
 'j': 20,
 'k': 2,
 'l': 4,
 'm': 5,
 'n': 18,
 'o': 11,
 'p': 9,
 'r': 19,
 's': 15,
 't': 0,
 'u': 22,
 'v': 23,
 'w': 24,
 'y': 1,
 'z': 17,
 '|': 7}

In [ ]:
import json

with open("vocab.json", "w") as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer(
    "./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|"
)

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1,
    sampling_rate=16000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=True,
)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained("/content/drive/MyDrive/container_0/wav2vec2-large-xlsr-ja")

In [ ]:
import torchaudio
import librosa
import numpy as np

resampler = torchaudio.transforms.Resample(48_000, 16_000)

def speech_file_to_array(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    batch["sampling_rate"] = 16_000
    batch["target_text"] = batch["text"]
    return batch

train_dataset = train_dataset.map(
    speech_file_to_array, remove_columns=train_dataset.column_names
)
test_dataset = test_dataset.map(
    speech_file_to_array, remove_columns=test_dataset.column_names
)

  0%|          | 0/4500 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

In [ ]:
# https://discuss.huggingface.co/t/wav2vec2-0-memory-issue/4868
def remove_long_common_voicedata(dataset, max_seconds=6):
    #convert pyarrow table to pandas
    dftest = dataset.to_pandas()

    #find out length of input_values
    dftest['len'] = dftest['speech'].apply(len)

    #for wav2vec training we already resampled to 16khz
    #remove data that is longer than max_seconds (6 seconds ideal)

    maxLength = max_seconds*16000 
    dftest = dftest[dftest['len'] < maxLength]
    dftest = dftest.drop('len', 1)

    #convert back to pyarrow table to use in trainer
    return dataset.from_pandas(dftest)


train_dataset = remove_long_common_voicedata(train_dataset)
test_dataset = remove_long_common_voicedata(test_dataset)

In [ ]:
len(train_dataset), len(test_dataset)

(3501, 394)

In [ ]:
train_dataset[:10]['sampling_rate']

[16000, 16000, 16000, 16000, 16000, 16000, 16000, 16000, 16000, 16000]

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_dataset))

ipd.Audio(data=np.asarray(train_dataset[rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(
        batch["speech"], sampling_rate=batch["sampling_rate"][0]
    ).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

train_dataset = train_dataset.map(
    prepare_dataset,
    remove_columns=train_dataset.column_names,
    batch_size=4,
    batched=True,
)
test_dataset = test_dataset.map(
    prepare_dataset,
    remove_columns=test_dataset.column_names,
    batch_size=4,
    batched=True,
)

  0%|          | 0/876 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


  0%|          | 0/99 [00:00<?, ?ba/s]

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from transformers import Wav2Vec2Processor


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [
            {"input_values": feature["input_values"]} for feature in features
        ]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
!pip install jiwer

     |████████████████████████████████| 50 kB 4.4 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149862 sha256=345b6b62d789b7f71bd3c9827dc41cefbe4b964f10113bf3db9fc1cababde807
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
from datasets import load_metric

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

In [ ]:
from transformers import Wav2Vec2ForCTC
from transformers import TrainingArguments, Trainer

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

model.freeze_feature_extractor()

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/container_0/ckpts/",
    logging_dir="/content/drive/MyDrive/container_0/runs/",
    group_by_length=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=30,
    fp16=True,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=1000,
    learning_rate=4e-4,
    warmup_steps=int(0.1 * len(train_dataset)),  # 10%
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'quantizer.weight_proj.bias', 'quantizer.weight_proj.weight', 'project_hid.weight', 'quantizer.codevectors', 'project_hid.bias', 'project_q.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to u

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 3501
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 6570
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, round

Step,Training Loss,Validation Loss,Wer
1000,1.193200,0.048715,0.721769
2000,0.143600,0.041763,0.688009


***** Running Evaluation *****
  Num examples = 394
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/container_0/ckpts/checkpoint-1000
Configuration saved in /content/drive/MyDrive/container_0/ckpts/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/container_0/ckpts/checkpoint-1000/pytorch_model.bin
Configuration saved in /content/drive/MyDrive/container_0/ckpts/checkpoint-1000/preprocessor_config.json
Deleting older checkpoint [/content/drive/MyDrive/container_0/ckpts/checkpoint-5000] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.di

In [ ]:
trainer.save_model("/content/drive/MyDrive/container_0/wav2vec2-large-xlsr-ja")